In [15]:
import numpy as np
import pandas as pd
import requests 
import json
from pandas.io.json import json_normalize


# URL
# https://waterdata.usgs.gov/pa/nwis/uv?format=rdb&site_no=01571500&period=&begin_date=2021-01-01&end_date=2021-01-20


In [16]:
FORMAT = 'json' # rdb
SITE_NUMBER = '01571500' # Yellow Breeches Creek near Camp Hill, PA
BEGIN_DATE = '2020-01-01' # 1985-10-01 / 2007-10-01
END_DATE = '2020-02-01' # 2021-01-18 / 2021-01-18

EXPORT_PATH = ''
EXPORT_FILE = ''

In [17]:
# url = 'https://nwis.waterdata.usgs.gov/pa/nwis/uv/?format={}&site_no={}&period=&begin_date={}&end_date={}'.format(FORMAT, SITE_NUMBER, BEGIN_DATE, END_DATE)
url = 'https://nwis.waterservices.usgs.gov/nwis/iv/?site={}&format={}&startDT={}&endDT={}'.format(SITE_NUMBER, FORMAT, BEGIN_DATE, END_DATE)
url

'https://nwis.waterservices.usgs.gov/nwis/iv/?site=01571500&format=json&startDT=2020-01-01&endDT=2020-02-01'

In [18]:
# Get Request
response = requests.get(url).json()
# response


In [19]:
foobar = response['value']['timeSeries'][0]['variable']
foobar

{'variableCode': [{'value': '00060',
   'network': 'NWIS',
   'vocabulary': 'NWIS:UnitValues',
   'variableID': 45807197,
   'default': True}],
 'variableName': 'Streamflow, ft&#179;/s',
 'variableDescription': 'Discharge, cubic feet per second',
 'valueType': 'Derived Value',
 'unit': {'unitCode': 'ft3/s'},
 'options': {'option': [{'name': 'Statistic', 'optionCode': '00000'}]},
 'note': [],
 'noDataValue': -999999.0,
 'variableProperty': [],
 'oid': '45807197'}

In [20]:
time_series_dfs = []
for i, series in enumerate(response['value']['timeSeries']):
    print("Series: ", str(i))
    print("Variable Name: ", series['variable']['variableName'])
    print("Variable Description: ", series['variable']['variableDescription'])
    print("Variable Units: ", series['variable']['unit']['unitCode'])
    
    # Create Data Frame
    series_df = json_normalize(series['values'][0]['value'])
    
    # Rename Value Column to be the Variable Name
    new_value_name = series['variable']['variableName'].split(',')[0].replace(' ', '_').lower()
    series_df.rename(columns={'value': new_value_name}, inplace=True)
    # Add Unit Column
    series_df[new_value_name+'_units'] = series['variable']['unit']['unitCode']
    # Drop Qualifiers Columns
    series_df.drop(['qualifiers'], axis=1, inplace=True)
    
    print(series_df.head())
    print('\n')
    
    # Push to Array of DF
    time_series_dfs.append(series_df)
    

Series:  0
Variable Name:  Streamflow, ft&#179;/s
Variable Description:  Discharge, cubic feet per second
Variable Units:  ft3/s
  streamflow                       dateTime streamflow_units
0        311  2020-01-01T00:00:00.000-05:00            ft3/s
1        311  2020-01-01T00:15:00.000-05:00            ft3/s
2        308  2020-01-01T00:30:00.000-05:00            ft3/s
3        311  2020-01-01T00:45:00.000-05:00            ft3/s
4        308  2020-01-01T01:00:00.000-05:00            ft3/s


Series:  1
Variable Name:  Gage height, ft
Variable Description:  Gage height, feet
Variable Units:  ft
  gage_height                       dateTime gage_height_units
0        1.69  2020-01-01T00:00:00.000-05:00                ft
1        1.69  2020-01-01T00:15:00.000-05:00                ft
2        1.68  2020-01-01T00:30:00.000-05:00                ft
3        1.69  2020-01-01T00:45:00.000-05:00                ft
4        1.68  2020-01-01T01:00:00.000-05:00                ft




<ipython-input-20-e10b28ef41c5>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  series_df = json_normalize(series['values'][0]['value'])
<ipython-input-20-e10b28ef41c5>:9: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  series_df = json_normalize(series['values'][0]['value'])


In [21]:
# merge the Time Series DataFrame
df = pd.merge(time_series_dfs[0], time_series_dfs[1], on='dateTime')
df.set_index('dateTime', inplace=True)
df.index.name = None
df.head()

,streamflow,streamflow_units,gage_height,gage_height_units
2020-01-01T00:00:00.000-05:00,311,ft3/s,1.69,ft
2020-01-01T00:15:00.000-05:00,311,ft3/s,1.69,ft
2020-01-01T00:30:00.000-05:00,308,ft3/s,1.68,ft
2020-01-01T00:45:00.000-05:00,311,ft3/s,1.69,ft
2020-01-01T01:00:00.000-05:00,308,ft3/s,1.68,ft


In [ ]:
# Add Export 